In [2]:
import random
import numpy as np

In [3]:
#funktionen die man für RSA braucht


#zufallszahl mit n stellen generieren
def random_number_N_digits(n):
    start = pow(10, n-1)
    end = pow(10, n)-1
    return random.randint(start, end)

#test ob zahl p prim ist
def fermat_test(possible_p, num_of_tests):
    for i in range(num_of_tests):
        random_int = random.randint(2, possible_p)
        if pow(random_int, possible_p - 1, possible_p) != 1:
            return False
    return True

#besserer test ob p prim ist
def miller_rabin(possible_p):
    d = possible_p - 1
    r = 0
    while d % 2 == 0:
        d //= 2
        r += 1
    a = random.randint(2, possible_p-1)
    x = pow(a, d, possible_p)
    if x == 1 or x == possible_p - 1:
        return True
    while r > 1:
        x = pow(x, 2, possible_p)
        if x == 1:
            return False
        if x == possible_p - 1:
            return True
        r -= 1
    return False

#test ob num_of_tests mal ob possible_p prim ist
def is_prime(possible_p, num_of_tests):
    if possible_p == 2:
        return True

    if possible_p % 2 == 0:
        return False

    for i in range(num_of_tests):
        if miller_rabin(possible_p) == False:
            return False
    return True

#zufalls prim mit n stellen generieren
def make_prime(n):
    run = True
    while run == True:
        p1 = random_number_N_digits(n)
        if is_prime(p1, 8) == True:
            return(p1)

#größten gemeinsamen teiler von p1 & p2 finden
def ggt(p1, p2):
    if p1 == p2:
        return p1

    if p1 > p2:
        a = p1
        b = p2

    if p2 > p1:
        a = p2
        b = p1

    run=True

    while run == True:
        teiler = a // b
        rest = a % b
        a = b
        
        if rest == 0:
            return(b)
            run = False

        b = rest

#phi von n finden
def phi(n):
    unteiler = 0

    for i in range(1, n + 1):
        if ggt(n, i) == 1:
            unteiler += 1

    return unteiler

#erweiterter euklidischer algorithmus von p1 & p2
def erweiterter_euklid(p1, p2):
    if p1 > p2:
        a = p1
        b = p2

    if p2 > p1:
        a = p2
        b = p1
    
    eu = np.array([[0, 0, 0, 0, 0, 0]], dtype=object)

    phase2 = False
    run = True
    eui =- 1

    while run == True:
        eui += 1

        teiler = a // b
        rest = a % b

        eu[eui,0] = a
        eu[eui,1] = b
        eu[eui,2] = teiler
        eu[eui,3] = rest

        a = b
        if rest> 0:
            newrow = np.array([0,0,0,0,0,0])
            eu = np.append(eu, [newrow], axis = 0)
            
        if rest == 0:
            run = False
            phase2 = True
        
        b = rest
    
    num_rows, num_cols = eu.shape
    i2 = num_rows - 1

    while phase2 == True:
        if i2 >- 1:
            if i2 == num_rows-1:
                eu[i2, 4] = 0
                eu[i2, 5] = 1

            elif i2 <= num_rows - 2:
                eu[i2, 4] = eu[i2+1, 5]
                eu[i2, 5] = eu[i2+1, 4] - (eu[i2, 2] * eu[i2+1, 5])
            
        else:
            phase2 = False
            return(eu)
        i2 -= 1     

In [4]:
#generierung der keys


#nimmt pprim & qprim als input und gibt e,n & phi(n) als output 
def create(pprim, qprim):
    n = qprim * pprim
    phin = (qprim - 1) * (pprim - 1)
    run = True

    while run == True:
        e = random.randint(2, phin)
        if ggt(e, phin) == 1:
            run = False
    return(e, n, phin)

#nimmt e,n & phi(n) als input und gibt e & n als output
def publickey(e, n, phin):
    return(e, n)

#nimmt e,n & phi(n) als input und gibt d & n als output
def privkey(e, n, phin):
    dtable = erweiterter_euklid(e, phin)
    d = dtable[0, 5]
    k = dtable[0, 4]
    r = (e * (d)) + (phin * k)
    return(d, n)

#nimmt qprim & pprim als input und gibt private und publickey als output
def make_keys(qprim, pprim):
    zwisch = create(qprim, pprim)
    private = privkey(zwisch[0], zwisch[1], zwisch[2])
    public = publickey(zwisch[0], zwisch[1], zwisch[2])
    return(private, public)

#nimmt n als input und gibt private & publickey mit qprim & pprim mit n stellen als output
def keys_with_n_digit_primes(n):
    return make_keys(make_prime(n), make_prime(n))
 

 #nimmt einen schlüssel und eine zahl als input, gibt eine verschlüsselte/ entschlüsselte zahl als output
def crypt_rsa(input, key):
    power = int(key[0])
    modulo = int(key[1])
    w = pow(input, power, modulo)
    return w 

In [5]:
#erstellt die keys

keys = keys_with_n_digit_primes(200)

In [6]:
#funktionen die zum verschlüsseln benutzt werden

#nimmt einen string als input und gibt eine liste von zahlen als output 
def encrypt_text(input):
    encrypted_text_array = []
    for i in input:
        encrypted_text_array.append((crypt_rsa(ord(i), keys[1])))
    return encrypted_text_array

#nimmt eine liste von zahlen als input und gibt einen string zurück
def decrypt_text(input):
    decrypted_text_string= ""
    for i in input:
        decrypted_text_string += str(chr(crypt_rsa(i, keys[0])))
    return decrypted_text_string

In [7]:
print(make_prime(400))

9737982749810932251845187060977817362914572655675227286366456590960858881814800637722641864840160102028304375241095619689231645356886522412710733385976248816533791455654815889563686995306185942031330285072294301486631730369882829428637896364601408449416825009697664419972813958644963870299154878521198970186935695420772364873331296465883518277101070621142431984155032904663680030424101486750606707711


In [9]:
ciphertext = decrypt_text(encrypt_text("Test Text"))
print(ciphertext)

Test Text
